In [1]:
import helper
import matplotlib.pyplot as plt
from keras.applications import *
from keras.callbacks import EarlyStopping
import os

Using TensorFlow backend.


In [2]:
#设置各种参数
train_path = ['./data/train1/cat', './data/train1/dog']
test_path ='./data/test1/test1'
img_size =(299,299)
layer_num = 630
model_image ='./models/model_image_fine_tuning_inceptionresnetv2_0401.png'
model_weights_file = './models/weights_fine_tuning_inceptionresnetv2_with_outliers_0401.h5'
template_csv_path = './predicts/sample_submission.csv'
target_csv_path = './predicts/pred_fine_tuning_inceptionresnetv2_with_outliers_0401.csv'
MODEL = inception_resnet_v2.InceptionResNetV2
preprocess_func = inception_resnet_v2.preprocess_input

In [3]:
#获取训练集数据
X_train, y_train, image_files= helper.get_train_input_from_folder_with_subclasses(train_path, img_size, lambda_func=preprocess_func)
print("finished")

(25000, 299, 299, 3)
(25000, 1)
finished
finished


In [4]:
#构造模型，锁定base_model所有层
model = helper.get_fine_tuning_first_model(MODEL)

#可视化模型
#helper.visualize_model(model, model_image)
print("finished")

start
(?, ?, ?, 3)
(?, ?, ?, 1536)
finished


In [5]:
print("start")
#第一次训练新添加层权重
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(X_train, y_train, batch_size=128, epochs=8, validation_split=0.2, callbacks=[early_stopping])
print("finished")

start
Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 130s 7ms/step - loss: 0.2124 - acc: 0.9301 - val_loss: 0.1364 - val_acc: 0.9518
Epoch 2/8
20000/20000 [==============================] - 123s 6ms/step - loss: 0.0882 - acc: 0.9732 - val_loss: 0.0962 - val_acc: 0.9712
Epoch 3/8
20000/20000 [==============================] - 123s 6ms/step - loss: 0.0734 - acc: 0.9756 - val_loss: 0.0653 - val_acc: 0.9808
Epoch 4/8
20000/20000 [==============================] - 123s 6ms/step - loss: 0.0652 - acc: 0.9781 - val_loss: 0.0711 - val_acc: 0.9780
Epoch 5/8
20000/20000 [==============================] - 123s 6ms/step - loss: 0.0644 - acc: 0.9774 - val_loss: 0.0921 - val_acc: 0.9720
Epoch 6/8
20000/20000 [==============================] - 123s 6ms/step - loss: 0.0616 - acc: 0.9791 - val_loss: 0.1089 - val_acc: 0.9654
finished


In [6]:
print("start")
#放开若干层权重，再次训练
model = helper.get_fine_tuning_second_model(model, layer_num)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
model.fit(X_train, y_train, batch_size=128, epochs=50, validation_split=0.2, callbacks=[early_stopping])
print("finished")

start
Train on 20000 samples, validate on 5000 samples
Epoch 1/50
20000/20000 [==============================] - 144s 7ms/step - loss: 0.0596 - acc: 0.9797 - val_loss: 0.0348 - val_acc: 0.9924
Epoch 2/50
20000/20000 [==============================] - 137s 7ms/step - loss: 0.0546 - acc: 0.9805 - val_loss: 0.0324 - val_acc: 0.9918
Epoch 3/50
20000/20000 [==============================] - 137s 7ms/step - loss: 0.0509 - acc: 0.9834 - val_loss: 0.0306 - val_acc: 0.9916
Epoch 4/50
20000/20000 [==============================] - 137s 7ms/step - loss: 0.0508 - acc: 0.9819 - val_loss: 0.0297 - val_acc: 0.9918
Epoch 5/50
20000/20000 [==============================] - 137s 7ms/step - loss: 0.0497 - acc: 0.9827 - val_loss: 0.0291 - val_acc: 0.9922
Epoch 6/50
20000/20000 [==============================] - 137s 7ms/step - loss: 0.0491 - acc: 0.9832 - val_loss: 0.0282 - val_acc: 0.9920
Epoch 7/50
20000/20000 [==============================] - 137s 7ms/step - loss: 0.0483 - acc: 0.9826 - val_loss: 0.02

In [7]:
#保存模型参数
model.save_weights(model_weights_file)
del X_train
del y_train
print("finished")

finished


In [8]:
print("start")
#获取测试数据和对应文件
X_test, test_files = helper.get_input_from_folder_with_image_files(test_path, img_size, lambda_func=preprocess_func)

#获取文件basename
image_file_names = [os.path.basename(path) for path in test_files]

#预测并保存预测结果到csv
helper.predict_and_update_to_csv(model, X_test, image_file_names, template_csv_path, target_csv_path)

print("finished")

start
finished
12500/12500 [==============================] - 71s 6ms/step
finished
